In [1]:
# Why do I use notebook right inside the feast_feature_store folder?
# because If I move it outside the repo folder, I'll get the 'FileNotFound' error.
# looks like Feast rn cannot understand that notebooks might call the repo outside of the folder -> doesnt append paths.
# The workaround is to use push server and get historical features as online features

In [70]:
import feast
import pickle
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
entity_df = pd.read_parquet('data/titanic_train_file_source.parquet')

In [71]:
entity_df.shape

(2663, 13)

In [46]:
import feast
import pickle
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
entity_df = pd.read_parquet('data/titanic_train_file_source.parquet')

fs = feast.FeatureStore(repo_path=".")
entity_df['event_timestamp'] = datetime(2022,1,15,2,59,50)
training_df = fs.get_historical_features(
    entity_df=entity_df[['PassengerId', 'event_timestamp']],
    features=[
        "passenger_stats:Survived",
        "passenger_stats:Pclass",
        "passenger_stats:Fare",
    ],
).to_df()

/Users/sfeda/Projects/feature_store_pet/.venv/lib/python3.10/site-packages/feast/repo_config.py:211: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


In [47]:
test =
training_df = fs.get_historical_features(
    entity_df=entity_df[['PassengerId', 'event_timestamp']],
    features=[
        "passenger_stats:Survived",
        "passenger_stats:Pclass",
        "passenger_stats:Fare",
    ],
).to_df()

In [69]:
fs.get_online_features(
    entity_rows=[{'PassengerId': 1800},
                 {'PassengerId': 3}],
    features=[
        "passenger_stats:Survived",
        "passenger_stats:Pclass",
        "passenger_stats:Fare",
    ],
).to_df()

,PassengerId,Survived,Fare,Pclass
0,1800,0.0,85.0,1.0
1,3,NaN,NaN,NaN


In [7]:
cols_to_drop = ['PassengerId', 'event_timestamp', 'Survived']
X = training_df.drop(columns=cols_to_drop)
y = training_df['Survived']
model = RandomForestClassifier(random_state=14).fit(X, y)


In [10]:
with open('passenger_classifier.pkl', 'wb') as f:
    pickle.dump(model, f)